# imports

In [1]:
import numpy as np
import tensorflow as tf

import pandas as pd
import os
from argparse import Namespace

2023-07-14 11:04:51.883967: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# config

In [2]:
if os.environ.get("KAGGLE_KERNEL_RUN_TYPE", ""):
    BASE_DIR = '/kaggle/input/google-research-identify-contrails-reduce-global-warming'
else:
    BASE_DIR =  'data'

In [3]:
configs = Namespace(
        base_dir= BASE_DIR,
        batch_size= 32,
        learning_rate= 0.001,
        epochs= 10,
)

# Helper functions

In [4]:
df_train_idx = pd.DataFrame({'idx': os.listdir(f'{configs.base_dir}/train')})
df_validation_idx = pd.DataFrame({'idx': os.listdir(f'{configs.base_dir}/validation')})

In [5]:


_T11_BOUNDS = (243, 303)
_CLOUD_TOP_TDIFF_BOUNDS = (-4, 5)
_TDIFF_BOUNDS = (-4, 2)


def get_band_images(idx, parent_folder, band):
    return np.load(os.path.join(configs.base_dir, parent_folder, idx, f'band_{band}.npy'))


def normalize_range(data, bounds):
    """Maps data to the range [0, 1]."""
    return (data - bounds[0]) / (bounds[1] - bounds[0])


def get_ash_color_images(idx, parent_folder, get_mask_frame_only=False):
    band11 = get_band_images(idx, parent_folder, '11')
    band14 = get_band_images(idx, parent_folder, '14')
    band15 = get_band_images(idx, parent_folder, '15')
    
    if get_mask_frame_only:
        band11 = band11[:,:,4]
        band14 = band14[:,:,4]
        band15 = band15[:,:,4]

    r = normalize_range(band15 - band14, _TDIFF_BOUNDS)
    g = normalize_range(band14 - band11, _CLOUD_TOP_TDIFF_BOUNDS)
    b = normalize_range(band14, _T11_BOUNDS)
    false_color = np.clip(np.stack([r, g, b], axis=2), 0, 1)
    return false_color

def get_mask_image(idx, parent_folder):
    return np.load(os.path.join(configs.base_dir, parent_folder, idx, 'human_pixel_masks.npy'))

In [6]:
class ContrailsAshDataset():
    def __init__(self, parent_folder):
        self.df_idx = pd.DataFrame({'idx': os.listdir(f'{configs.base_dir}/{parent_folder}')})
        self.parent_folder = parent_folder

    def __len__(self):
        return len(self.df_idx)

    def __getitem__(self, idx):
        image_id = str(self.df_idx.iloc[idx]['idx'])
        images = np.reshape(get_ash_color_images(image_id, self.parent_folder, get_mask_frame_only=False), (256, 256, 24)).astype(np.float32)
        mask = get_mask_image(image_id, self.parent_folder).astype(np.float32)
      
        return images, mask

In [7]:
AUTOTUNE = tf.data.AUTOTUNE

class GetDataloader():
    def __init__(self, args):
        self.args = args
        
    def dataloader(self, dataset, data_type='train'):
        # Create dataset from the ContrailsAshDataset object
        dataloader = tf.data.Dataset.from_generator(
            lambda: dataset,
            output_signature=(
                tf.TensorSpec(shape=(256, 256, 24), dtype=tf.float32),
                tf.TensorSpec(shape=(256, 256, 1), dtype=tf.float32)
            )
        )
        
        # Shuffle if it's for training
        if data_type == 'train':
            dataloader = dataloader.shuffle(self.args.batch_size)

        # Add general transformations
        dataloader = (
            dataloader
            .batch(self.args.batch_size)
            .prefetch(AUTOTUNE)
        )

        return dataloader



# load data

In [8]:
dataset_train = ContrailsAshDataset('train')

train_dataloader = GetDataloader(configs).dataloader(dataset_train, data_type="train")

train_dataloader

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 24), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 1), dtype=tf.float32, name=None))>

In [9]:
# print shape of one batch
for images, masks in train_dataloader.take(1):
    print(images.shape)
    print(masks.shape)

(6, 256, 256, 24)
(6, 256, 256, 1)


2023-07-14 11:05:02.854893: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-07-14 11:05:02.855226: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [10]:
dataset_val = ContrailsAshDataset('validation')

val_dataloader = GetDataloader(configs).dataloader(dataset_val, data_type="val")

val_dataloader

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 24), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 1), dtype=tf.float32, name=None))>

# model

## U-Net

In [11]:
# define U-Net model

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Conv2DTranspose, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import backend as K
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.metrics import MeanIoU

def iou_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou

def iou_coef_loss(y_true, y_pred):
    return 1 - iou_coef(y_true, y_pred)

def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    dice = K.mean((2. * intersection + smooth)/(union + smooth), axis=0)
    return dice

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

class UNet:

    def __init__(self, input_shape):
        self.input_shape = input_shape

    def _conv_block(self, input_tensor, n_filters, kernel_size=3):
        x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), padding="same")(input_tensor)
        x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), padding="same")(x)
        return x

    def resnet_encoder(self, input_tensor, filters):
        skip= self._conv_block(input_tensor, filters)
        x = MaxPooling2D((2, 2))(skip)

        return x, skip
    
    def resnet_decoder(self, input_tensor, skip_tensor, filters):
        x = Conv2DTranspose(filters=filters, kernel_size=(3, 3), strides=(2, 2), padding="same")(input_tensor)
        x = concatenate([x, skip_tensor])
        x = self._conv_block(x, filters)
        return x
    
    def bottleneck(self, input_tensor, filters):
        x = self._conv_block(input_tensor, filters)
        return x

    def build(self):

        input_tensor = Input(shape=self.input_shape)
        x = input_tensor

        

        # Encoder
        x, skip1 = self.resnet_encoder(x, 64)
        # Add batch normalization layer
        x = tf.keras.layers.BatchNormalization()(x)
        x, skip2 = self.resnet_encoder(x, 128)
        x = Dropout(0.3)(x)
        x, skip3 = self.resnet_encoder(x, 256)
        x, skip4 = self.resnet_encoder(x, 512)
        x = Dropout(0.3)(x)

        # Bottleneck
        x = self.bottleneck(x, 1024)

        # Decoder
        x = self.resnet_decoder(x, skip4, 512)
        x = Dropout(0.3)(x)
        x = self.resnet_decoder(x, skip3, 256)
        x = self.resnet_decoder(x, skip2, 128)
        x = Dropout(0.3)(x)
        x = self.resnet_decoder(x, skip1, 64)

        # Output
        output_tensor = Conv2D(filters=1, kernel_size=(1, 1), activation="sigmoid")(x)

        model = Model(input_tensor, output_tensor)
        return model

    # add dice_coef to loss
    # def compile(self, optimizer, loss, metrics):
    #     super().compile(
    #         optimizer=optimizer
    #     )

# create model

model = UNet((256, 256, 24)).build()

model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss=iou_coef_loss,
    metrics=[iou_coef, dice_coef]
)

model.summary()



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 2  0           []                               
                                4)]                                                               
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  13888       ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 64  36928       ['conv2d[0][0]']                 
                                )                                                             

In [12]:
# model = UNet((256, 256, 24)).build()

# model.summary()

# # train model

# model.compile(optimizer=Adam(learning_rate=configs.learning_rate), loss=iou_coef_loss, metrics=[iou_coef, dice_coef])

callbacks = [
    EarlyStopping(patience=10, verbose=1, restore_best_weights=True, monitor='loss'),
    ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

# history = model.fit(train_dataloader, epochs=configs.epochs, callbacks=callbacks, verbose=1)

history = model.fit(train_dataloader, epochs=4, verbose=1, validation_data=val_dataloader)

Epoch 1/4


2023-07-14 11:05:15.187600: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-07-14 11:05:15.187848: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-07-14 11:05:18.465037: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 100663296 exceeds 10% of free system memory.
2023-07-14 11:05:18.545534: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 100663296 exceeds 10% of free system memory.
2023-07-14 11:05:19.958309: W tensorflow/tsl/framework/cpu_allocator

      1/Unknown - 18s 18s/step - loss: 0.9888 - iou_coef: 0.0112 - dice_coef: 0.0215

2023-07-14 11:05:33.845380: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1/1 [==============================] - 21s 21s/step - loss: 0.9888 - iou_coef: 0.0112 - dice_coef: 0.0215 - val_loss: 1.0000 - val_iou_coef: 2.5391e-05 - val_dice_coef: 2.5391e-05
Epoch 2/4
1/1 [==============================] - 14s 14s/step - loss: 0.9867 - iou_coef: 0.0133 - dice_coef: 0.0253 - val_loss: 1.0000 - val_iou_coef: 1.8966e-05 - val_dice_coef: 1.8966e-05
Epoch 3/4
1/1 [==============================] - 12s 12s/step - loss: 0.9862 - iou_coef: 0.0138 - dice_coef: 0.0262 - val_loss: 0.9999 - val_iou_coef: 6.5883e-05 - val_dice_coef: 6.5883e-05
Epoch 4/4
1/1 [==============================] - 13s 13s/step - loss: 0.9854 - iou_coef: 0.0146 - dice_coef: 0.0270 - val_loss: 0.9990 - val_iou_coef: 9.7950e-04 - val_dice_coef: 9.7950e-04


# save the model

In [ ]:
# model.save('model.h5')

# test

In [13]:
df_test_idx = pd.DataFrame({'idx': os.listdir(f'{configs.base_dir}/test')})

In [14]:
parent_dir = "test"

# get test images in the form of batch
def get_test_images(test_dir):
    test_images = []
    for idx in os.listdir(os.path.join(configs.base_dir, test_dir)):
        # image_path = os.path.join(test_dir, idx)
        image = get_ash_color_images(idx, test_dir, get_mask_frame_only=False)
        test_images.append(image)
    test_images = np.array(test_images)
    return np.reshape(test_images, (len(test_images), 256, 256, 24))


test_images = get_test_images("test")

predictions = model.predict(test_images)


1/1 [==============================] - 1s 1s/step


In [15]:
def create_mask(image, threshold):
    mask = tf.math.greater(image, threshold)
    return tf.cast(mask, tf.float32)

masked_pred = create_mask(predictions, 0.5)

# rle encoding

In [16]:
#source https://www.kaggle.com/code/inversion/contrails-rle-submission?scriptVersionId=128527711&cellId=4

def rle_encode(x, fg_val=1):
    """
    Args:
        x:  numpy array of shape (height, width), 1 - mask, 0 - background
    Returns: run length encoding as list
    """

    dots = np.where(
        x.T.flatten() == fg_val)[0]  # .T sets Fortran order down-then-right
    run_lengths = []
    prev = -2
    for b in dots:
        if b > prev + 1:
            run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths


def list_to_string(x):
    """
    Converts list to a string representation
    Empty list returns '-'
    """
    if x: # non-empty list
        s = str(x).replace("[", "").replace("]", "").replace(",", "")
    else:
        s = '-'
    return s

len(list_to_string(rle_encode(np.array(masked_pred)[0])))

3349

In [17]:
# make a submission dataframe with record_id as index and rle string as column

submission_df = pd.DataFrame(columns=['encoded_pixels'])
submission_df.index.name = 'record_id'

for i in range(len(masked_pred)):
    submission_df.loc[df_test_idx.iloc[i]["idx"], 'encoded_pixels'] = list_to_string(rle_encode(np.array(masked_pred)[i]))

submission_df

,encoded_pixels
record_id,
1000834164244036115,17 73 101 1 103 1 105 1 107 1 109 37 270 78 35...
1002653297254493116,1 25 51 37 98 8 145 137 306 38 353 10 400 139 ...


In [ ]:
# submission_df.to_csv('submission.csv')